## Chapter 8
# Vibrating Systems

## Stretched Circular Membranes

This is based off of the discussion of the modes of drum head vibrations (p266-270), which can be modelled with Bessel functions, where the radial amplitude of the $(m-1,n)^{th}$ mode is given by the $x$ value ranging from $0$ to the $n^{th}$ root of the $m^{th}$ order Bessel function.

There is no known simple equation for finding the roots of the Bessel functions, but `scipy` provides a table of many of the zero values directly.

The book gives the equation for the vibration pattern of the drum modes  on p268 as:

$J_m(rx)\cos(m\phi)\cos(tx)$,

where $m$ is the number of circular nodes in the mode, $r$ and $\phi$ are the polar coordinates of the point on the surface of the drum being evaluated, and $x$ is the value of the $nth$ root of the $m^{th}$-order Bessel function.

In [65]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import numpy as np
from mpl_toolkits.mplot3d.axes3d import Axes3D
# Bessel functions and their roots (zero crossings)
from scipy.special import jn, jn_zeros

# rows correspond to root number
# columns correspond to Bessel order
# (e.g. 2nd row, 1st column == 2nd root of 0-order Bessel function == "Mode 02")
fig = plt.figure(figsize=(16, 12))

bessel_order_max = 4
bessel_root_max = 3

axes = []
for x in range(bessel_order_max):
    axes.append([])
    for y in range(bessel_root_max):
        axes[x].append(fig.add_subplot(bessel_root_max, bessel_order_max, y * 4 + x + 1, projection='3d'))

dim = 50
r, phi = np.meshgrid(np.linspace(0, 1, dim), np.linspace(0, 2 * np.pi, dim))
x, y = r * np.cos(phi), r * np.sin(phi)

bessel_roots = np.array([jn_zeros(bessel_order, bessel_root_max) for bessel_order in range(bessel_order_max)])

jns = np.ndarray((bessel_order_max, bessel_root_max, dim, dim))
for m in range(bessel_order_max):
    for n in range(bessel_root_max):
        jns[m][n] = jn(m, bessel_roots[m, n] * r) * np.cos(m * phi)

frames_per_sec = 24
num_frames = frames_per_sec * 3

def animate(i):
    t = 2 * np.pi * i / num_frames
    for m in range(bessel_order_max):
        for n in range(bessel_root_max):
            z = jns[m,n] * np.sin(t)
            ax = axes[m][n]
            ax.cla()
            plot = ax.plot_surface(x, y, z, cmap='jet', vmin=-1, vmax=1)
            ax.set_zlim(-1, 1)
            ax.set_title('Mode %i%i' % (m, n + 1))
            ax.set_xticks([], [])
            ax.set_yticks([], [])
            ax.set_zticks([], [])

anim = animation.FuncAnimation(fig, animate, frames=num_frames, interval=1_000/frames_per_sec)
plt.close()
HTML(anim.to_html5_video())